In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Create image: 1 row x 100 pixels
img = np.zeros((1, 100, 3), dtype=np.uint8)

# First 20 pixels = red (mercury)
for i in range(20):
    img[0, i] = [0, 0, 255]  # Red in BGR

# Remaining pixels alternate black and white
for i in range(20, 100):
    img[0, i] = [255, 255, 255] if i % 2 == 0 else [0, 0, 0]

# Convert to HSV
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Define red range in HSV (two ranges for red)
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])

# Create masks for both red ranges
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
red_mask = cv2.bitwise_or(mask1, mask2)

# Show red mask
plt.imshow(red_mask, cmap='gray')
plt.title('Red Mercury Detection')
plt.axis('off')
plt.show()

# Find red mercury pixels
coords = cv2.findNonZero(red_mask)

if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    boxed_img = img.copy()
    cv2.rectangle(boxed_img, (x, y), (x + w, y + h), (0, 255, 0), 1)

    # Show final image with bounding box
    resized = cv2.resize(boxed_img, (1000, 100), interpolation=cv2.INTER_NEAREST)
    plt.imshow(cv2.cvtColor(resized, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title("Detected Mercury (Only Red)")
    plt.show()

    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])
    mercury_length = max_x - min_x + 1

    total_pixels = img.shape[1]
    temperature_range = 600
    pixels_per_degree = total_pixels / temperature_range
    temperature = mercury_length / pixels_per_degree

    print(f"Red mercury from column {min_x} to {max_x}")
    print(f"Mercury pixel length: {mercury_length}")
    print(f"Measured temperature: {temperature:.2f} °C")

else:
    print("No red mercury detected.")
